In [1]:
import os
import subprocess
import whisper
import torch
from pathlib import Path

In [4]:
# taken from https://github.com/deezer/spleeter/issues/927#issuecomment-2629364261 due to spleeter not being compatible with latest python version

# Set the root for pyenv to the desired directory
os.environ['PYENV_ROOT'] = '/vol/bitbucket/sg2121/.pyenv'

# Prepend pyenv's bin folder to PATH
os.environ['PATH'] = os.environ['PYENV_ROOT'] + '/bin:' + os.environ['PATH']

# Confirm pyenv is callable
!git clone https://github.com/pyenv/pyenv.git $PYENV_ROOT
!pyenv --version

# Install necessary dependencies
!apt-get install libffi-dev

# Install Python 3.8.13 to the specified location
!pyenv install 3.8.13

# Set the global Python version to 3.8.13
!pyenv global 3.8.13

# Install spleeter in the pyenv environment
!pyenv exec pip install spleeter

# Run spleeter
!pyenv exec spleeter


fatal: destination path '/vol/bitbucket/sg2121/.pyenv' already exists and is not an empty directory.
pyenv 2.5.5-1-g323af282
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
-> https://www.python.org/ftp/python/3.8.13/Python-3.8.13.tar.xz
Installing Python-3.8.13...
Installed Python-3.8.13 to /vol/bitbucket/sg2121/.pyenv/versions/3.8.13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 35.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.9 MB/s eta 0:00:000m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 39.2 MB/s eta 0:00:0

In [11]:
# Define the input and output paths
input_file = "/data/sg2121/fypdataset/dataset/normal_data/ai/segments/S01RN_segment_1.mp3"
output_folder = "/data/sg2121/fypdataset/dataset/test"

# Run the Spleeter command with pyenv exec
subprocess.run([
    "pyenv", "exec", "spleeter", "separate", 
    "-o", output_folder, 
    input_file
])


INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 2stems archive
INFO:spleeter:2stems model file(s) extracted
INFO:spleeter:File /data/sg2121/fypdataset/dataset/test/S01RN_segment_1/vocals.wav written succesfully
INFO:spleeter:File /data/sg2121/fypdataset/dataset/test/S01RN_segment_1/accompaniment.wav written succesfully


CompletedProcess(args=['pyenv', 'exec', 'spleeter', 'separate', '-o', '/data/sg2121/fypdataset/dataset/test', '/data/sg2121/fypdataset/dataset/normal_data/ai/segments/S01RN_segment_1.mp3'], returncode=0)

In [3]:
model = whisper.load_model("tiny")  # or "small", "medium", "large"
result = model.transcribe("/data/sg2121/fypdataset/dataset/test/S01RN_segment_1/vocals.wav")
print(result["text"])


AttributeError: module 'whisper' has no attribute 'load_model'

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = whisper.load_model("base", device=device)
result = model.transcribe("/data/sg2121/fypdataset/dataset/normal_data/ai/segments/S05RN_segment_1.mp3")
print(result["text"])

Using device: cuda
 To the left, to the right, everywhere we go As a wild night, hit the black on lies on me Got my hook in the back, can we roll it? One of the top, but they ain't got the heat I'm the realist and it's watching me on Brim's finnest throw, looking like a damn star In the club, are we on the fuck off From the hood to the fuck, they know my name Ain't no stomping this, we playing the game I'm pooping off, it's how you know it's I want my own real plan, we just f***ing big to the left


In [ ]:
# Load Whisper model once
model = whisper.load_model("small")  # or "small", "medium", "large"

# Root directory containing subfolders with vocals.wav
input_dir = "/data/sg2121/fypdataset/dataset/normal_data/human"
output_dir = "/data/sg2121/fypdataset/dataset/lyrics/human"

# Make sure output folder exists
os.makedirs(output_dir, exist_ok=True)

# Process each audio file
for filename in os.listdir(input_dir):
    if filename.endswith(".mp3"):
        input_path = os.path.join(input_dir, filename)

        # Define output path: song1.wav → song1_lyrics.txt (in output_dir)
        base_name = os.path.splitext(filename)[0]
        output_path = os.path.join(output_dir, f"{base_name}_lyrics.txt")

        # Skip if already processed
        if os.path.exists(output_path):
            print(f"Skipping (already exists): {output_path}")
            continue

        print(f"Transcribing: {input_path}")
        try:
            result = model.transcribe(input_path)
            with open(output_path, "w") as f:
                f.write(result["text"])
            print(f"Saved: {output_path}")
        except Exception as e:
            print(f"Error transcribing {filename}: {e}")


In [2]:
# Settings
input_dir = Path("/data/sg2121/fypdataset/dataset/normal_data/ai/segments")
output_dir = Path("/data/sg2121/fypdataset/dataset/lyrics/ai")
output_dir.mkdir(parents=True, exist_ok=True)

# Load Whisper model (auto uses GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("base", device=device)

# Loop through audio files
for audio_path in input_dir.glob("*.*"):  # supports .wav, .mp3, etc.
    output_path = output_dir / f"{audio_path.stem}_lyrics.txt"

    # Skip if already transcribed
    if output_path.exists():
        print(f"Skipping (already exists): {output_path.name}")
        continue

    print(f"Transcribing: {audio_path.name}")
    try:
        result = model.transcribe(str(audio_path), fp16=True)
        text = result["text"].strip()

        # Save transcript
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(text)

        print(f"Saved: {audio_path.name}")

    except Exception as e:
        print(f"Failed to transcribe {audio_path.name}: {e}")



Transcribing: S24RN_segment_1.mp3
Saved: S24RN_segment_1.mp3
Transcribing: U23RN_segment_1.mp3
Saved: U23RN_segment_1.mp3
Transcribing: S40RN_segment_2.mp3
Saved: S40RN_segment_2.mp3
Transcribing: S04RN_segment_2.mp3
Saved: S04RN_segment_2.mp3
Transcribing: U80RN_segment_2.mp3
Saved: U80RN_segment_2.mp3
Transcribing: S63RN_segment_2.mp3
Saved: S63RN_segment_2.mp3
Transcribing: S91RN_segment_1.mp3
Saved: S91RN_segment_1.mp3
Transcribing: S03RN_segment_1.mp3
Saved: S03RN_segment_1.mp3
Transcribing: U96RN_segment_1.mp3
Saved: U96RN_segment_1.mp3
Transcribing: U56RN_segment_2.mp3
Saved: U56RN_segment_2.mp3
Transcribing: U50RN_segment_1.mp3
Saved: U50RN_segment_1.mp3
Transcribing: U69RN_segment_1.mp3
Saved: U69RN_segment_1.mp3
Transcribing: U25RN_segment_1.mp3
Saved: U25RN_segment_1.mp3
Transcribing: S64RN_segment_2.mp3
Saved: S64RN_segment_2.mp3
Transcribing: U68RN_segment_2.mp3
Saved: U68RN_segment_2.mp3
Transcribing: S18RN_segment_2.mp3
Saved: S18RN_segment_2.mp3
Transcribing: U19RN_segm